<a href="https://colab.research.google.com/github/hanna-bae/PBL/blob/main/model_test_fix.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

KcELECTRA 
- 학습으로 한국어 댓글(Korean Comments). 구어체와 같이 잘 정제되어있지 않은 데이터 셋 학습에 유리 


In [ ]:
!pip install transformers 

!git clone https://github.com/ZIZUN/korean-malicious-comments-dataset.git

import pandas as pd 
import matplotlib.pyplot as plt

import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
from sklearn.metrics import precision_recall_fscore_support, accuracy_score

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print('device:', device)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached transformers-4.29.2-py3-none-any.whl (7.1 MB)
fatal: destination path 'korean-malicious-comments-dataset' already exists and is not an empty directory.
device: cuda:0


In [ ]:
df = pd.read_csv('/content/korean-malicious-comments-dataset/Dataset.csv', sep='\t')
df.head()

,content,lable
0,이종석 한효주 나오는 드라마 이후로 드라마 안봤다. 2년전인가?? 좀 신선했었지. ...,0.0
1,씨바알..노무노무 술프노... 오늘 저녁은 꽂등심이다ㅠㅜ,0.0
2,짱깨 꺼라ㅡ패쓰,0.0
3,그들의 사생활 ~ 고인이된 설리를 위해서라도 모두 조용하길 지금 누굴 탓한다고 무슨...,1.0
4,아무리 법이 뭣같아도 무슨 자격으로 개인의 신상정보를 불특정 다수에게 공개하는지 도...,1.0


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   content  10000 non-null  object 
 1   lable    9975 non-null   float64
dtypes: float64(1), object(1)
memory usage: 156.4+ KB


In [ ]:
null_idx = df[df.lable.isnull()].index
df.loc[null_idx, 'content']

1602    응애 응애 엄마 저 맘에 안들죠? ........아들 ?? " 너 내가 우스워 보이...
1654           토니스타크 평소 "아이엠그루트"라는 유행어를 부러워했다는게 학계의 정설\t1
1992    "13일 현대차에 따르면 올 들어 국내 소비자들의 수입차 구매의향률이 3년 만에 하...
2920                 에이프릴이 한마디 합니다 "예쁜게 죄" 구하라님 "무기징역"\t1
3720          답글 글씨체를 봐라 저게 애새끼가 쓴거냐?"빨갱이새끼가 쓴거지 ㅁㅈㅎㅉㅉ\t0
3807    알겠다이기ㅋㅋ 딱 채찍쳐맞는거 좋아하는 한국식 마인드네. 노예마인드. 조금만 성공한...
3908           이래서 스스로 걸리거든 "죄인들이"~ㅎㅎㅎ 재미보고 털리고 그치~~~?\t0
4241    아버지는 내재된 악마들을 다룰 정신적 힘을 가지고 있지 않았다." 이 말한마디가 사...
4283    댓글 중 "선동 당해서 촞불든 개돼지 홍어들도 단죄를 받아야 할 공범자들이다"에10...
5000    스파이 제안받고 살해 안당하는 법1. 처음에 스파이 제안을 받았을때 "중국을 위해서...
5521    "국방부 "까지 ㅡㄱ ㅐ 엿같은 ㅈ ㅣ랄주댕이...좌빨에서 ㅡ인민군대로 ㅡ가려는건가...
5866    쌩뚱맞게 60대최반엌 치매라니 그것도 곱게 사는 사모님이- -" 알콜중독도 아니고 ...
6477    페미메퇘지쿵쾅년인 메갈페미들은 니들이 좋아하는 싫어요 ㄱㄱ제발부탁해~~"일반 여성"...
6538    아니 ㅆㅂ 그런 "카더라"가 넘쳐난다고 그거에 대해서 혹시 댓글게이는 뭔가 아는거 ...
6771    저 때 투니버스에서 코요태 짧게 인터뷰 했었는데 김종민이 "노래는 뭐 신지가 다 하...
6932               개 족 가튼 국방부의 "휴기연장콜센터"발족을 축하한다 ㅆ ㅂ..\t0
7199    민족적 자존심과 애국심을 갖고 국산품 이용합시다 . . . "겸손"한 마음으로 재산...
7252    아나운서는 

In [ ]:
df.loc[null_idx, 'lable'] = df.loc[null_idx, 'content'].apply(lambda x:x[-1])

df.loc[null_idx, 'content'] = df.loc[null_idx,'content'].apply(lambda x:x[:-2])

df.head()

,content,lable
0,이종석 한효주 나오는 드라마 이후로 드라마 안봤다. 2년전인가?? 좀 신선했었지. ...,0.0
1,씨바알..노무노무 술프노... 오늘 저녁은 꽂등심이다ㅠㅜ,0.0
2,짱깨 꺼라ㅡ패쓰,0.0
3,그들의 사생활 ~ 고인이된 설리를 위해서라도 모두 조용하길 지금 누굴 탓한다고 무슨...,1.0
4,아무리 법이 뭣같아도 무슨 자격으로 개인의 신상정보를 불특정 다수에게 공개하는지 도...,1.0


In [ ]:
df = df.astype({'lable':'int'})
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   content  10000 non-null  object
 1   lable    10000 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 156.4+ KB


In [ ]:
#train:test = 4:1
train_data = df.sample(frac=0.8, random_state=42)
test_data = df.drop(train_data.index)

In [ ]:
print('train data include duplicate : {}'.format(len(train_data)))
print('test data include duplicate : {}'.format(len(test_data)))

train_data.drop_duplicates(subset=['content'], inplace=True)
test_data.drop_duplicates(subset=['content'], inplace=True)

print('train data drop replication : {}'.format(len(train_data)))
print('test data drop replication : {}'.format(len(test_data)))


train data include duplicate : 8000
test data include duplicate : 2000
train data drop replication : 7992
test data drop replication : 2000


Tokenizing 
- 구두점이나 특수 문자를 단순 제외해서는 안됨.(Don't)
- 줄임말과 단어 내에 띄어쓰기가 있는 경우

In [ ]:
MODEL_NAME = 'beomi/KcELECTRA-base'
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

tokenized_train_sentences = tokenizer(
    list(train_data['content']),
    return_tensors='pt', #pytorch의 tensor 형태로 return 
    max_length=128,
    padding=True,
    truncation=True,
    add_special_tokens=True
)

In [ ]:
print(tokenized_train_sentences[0].ids)

[7727, 424, 497, 9927, 6413, 712, 3101, 372, 1566, 5898, 16745, 9129, 11079, 96, 1952, 7846, 3628, 299, 8668, 3819, 10134, 1744, 11079, 424, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3]


In [ ]:
tokenized_test_sentences = tokenizer(
    list(test_data['content']),
    return_tensors='pt',
    max_length=128,
    padding=True,
    truncation=True,
    add_special_tokens=True
)

In [ ]:
class CurseDataset(torch.utils.data.Dataset):
  def __init__(self, encodings, labels):
    self.encodings = encodings
    self.labels = labels

  def __getitem__(self, idx):
    item = {key:torch.tensor(val[idx]) for key, val in self.encodings.items()}
    item['labels'] = torch.tensor(self.labels[idx])
    return item
  
  def __len__(self):
    return len(self.labels)

    

In [ ]:
train_label = train_data['lable'].values
test_label = test_data['lable'].values

train_dataset = CurseDataset(tokenized_train_sentences, train_label)
test_dataset = CurseDataset(tokenized_test_sentences, test_label)

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained(MODEL_NAME, num_labels=2)
model.to(device)


Some weights of the model checkpoint at beomi/KcELECTRA-base were not used when initializing ElectraForSequenceClassification: ['discriminator_predictions.dense.bias', 'discriminator_predictions.dense_prediction.bias', 'discriminator_predictions.dense_prediction.weight', 'discriminator_predictions.dense.weight']
- This IS expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at beomi/KcELECTRA-base and are newly initialized: ['classifier.dense.weight', 'classifier.

ElectraForSequenceClassification(
  (electra): ElectraModel(
    (embeddings): ElectraEmbeddings(
      (word_embeddings): Embedding(30000, 768, padding_idx=3)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): ElectraEncoder(
      (layer): ModuleList(
        (0-11): 12 x ElectraLayer(
          (attention): ElectraAttention(
            (self): ElectraSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): ElectraSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): L

In [ ]:
!pip install --upgrade accelerate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms

In [ ]:
!pip uninstall -y transformers accelerate

Found existing installation: transformers 4.29.2
Uninstalling transformers-4.29.2:
  Successfully uninstalled transformers-4.29.2
Found existing installation: accelerate 0.20.0.dev0
Uninstalling accelerate-0.20.0.dev0:
  Successfully uninstalled accelerate-0.20.0.dev0


In [ ]:

!pip install git+https://github.com/huggingface/accelerate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/huggingface/accelerate to /tmp/pip-req-build-_b8ki3e1
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/accelerate /tmp/pip-req-build-_b8ki3e1
  Resolved https://github.com/huggingface/accelerate to commit bfa74e51d2af08221f5787d281d681ca9bceddd2
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for accelerate: filename=accelerate-0.20.0.dev0-py3-none-any.whl size=225536 sha256=be2935916b6fc3b677b466245818f5c01d8555448f54583a49a512cc1c33345b
  Stored in directory: /tmp/pip-ephem-wheel-cache-x_au81ie/wheels/f6/c7/9d/1b8a5ca8353d9307733bc719107acb67acdc95063bba749f26
Successfully built accelerate


In [ ]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir='./',
    num_train_epochs=10,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=64,
    logging_dir='./logs',
    logging_steps=500,
    save_total_limit=2,
)

In [ ]:
def compute_metric(pred):
  labels = pred.labels_ids
  preds = pred.predictions.argmax(-1)
  precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='binary')
  acc = accuracy_score(labels, preds)
  return {
      'accuracy' : acc,
      'f1' : f1,
      'precision': precision,
      'recall': recall
  }

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    tokenizer=tokenizer,
)

In [ ]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:407: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
<ipython-input-12-997d1d3604f3>:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key:torch.tensor(val[idx]) for key, val in self.encodings.items()}
You're using a PreTrainedTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss
500,0.415200
1000,0.353800
1500,0.260500
2000,0.269300
2500,0.190000
3000,0.224200
3500,0.194500
4000,0.161100
4500,0.110400
5000,0.106700


<ipython-input-12-997d1d3604f3>:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key:torch.tensor(val[idx]) for key, val in self.encodings.items()}
<ipython-input-12-997d1d3604f3>:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key:torch.tensor(val[idx]) for key, val in self.encodings.items()}
<ipython-input-12-997d1d3604f3>:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key:torch.tensor(val[idx]) for key, val in self.encodings.items()}
<ipython-input-12-997d1d3604f3>:7: UserWarning: To copy construct from a ten

TrainOutput(global_step=9990, training_loss=0.12811354352189255, metrics={'train_runtime': 2102.8419, 'train_samples_per_second': 38.006, 'train_steps_per_second': 4.751, 'total_flos': 5256958886092800.0, 'train_loss': 0.12811354352189255, 'epoch': 10.0})

test set


In [ ]:
!rm *.xlsx

from google.colab import files
uploaded = files.upload()


Saving 마이네임_1화.xlsx to 마이네임_1화.xlsx
Saving 마이네임_2화.xlsx to 마이네임_2화.xlsx
Saving 마이네임_3화.xlsx to 마이네임_3화.xlsx
Saving 마이네임_4화.xlsx to 마이네임_4화.xlsx
Saving 마이네임_5화.xlsx to 마이네임_5화.xlsx
Saving 마이네임_6화.xlsx to 마이네임_6화.xlsx
Saving 마이네임_7화.xlsx to 마이네임_7화.xlsx
Saving 마이네임_8화.xlsx to 마이네임_8화.xlsx


In [ ]:
import pandas as pd
import numpy as np

# Load the Excel data
for i in uploaded.keys():
    부부의세계 = pd.read_excel(i)
    부부의세계 = 부부의세계.drop(['Transliteration'], axis=1)
    부부의세계['label'] = np.nan  # Add 'label' column with NaN values

# Iterate over each row and update the 'label' values
for i, row in 부부의세계.iterrows():
    model.eval()

    token_ = tokenizer.encode_plus(
        row['Subtitle'], 
        truncation=True,
        add_special_tokens=True,
        max_length=128,
        padding="max_length",
        return_tensors='pt'
    )

    input_ids = token_['input_ids'].to(device)
    attention_mask = token_['attention_mask'].to(device)

    with torch.no_grad():
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        logits = outputs[0]
        logits = logits.detach().cpu()
        probabilities = torch.sigmoid(logits)
        result = probabilities.squeeze().tolist()

    부부의세계.loc[i, 'label'] = result[0] 

부부의세계.head()

,Time,Subtitle,label
0,49s,(선우) 자,0.010667
1,52s,학교,0.789786
2,54s,그대로 다녀도 괜찮겠어?,0.009678
3,56s,낯선 데 가서 적응하는 거보단,0.011575
4,59s,차라리 부딪쳐 보려고,0.009865


In [89]:
부부의세계.to_excel('부부의세계.xlsx', index=False)

In [ ]:
import pandas as pd
import numpy as np

# Load the Excel data
for i in uploaded.keys():
    수리남 = pd.read_excel(i)
    수리남 = 수리남.drop(['Transliteration'], axis=1)
    수리남['label'] = np.nan  # Add 'label' column with NaN values

# Iterate over each row and update the 'label' values
for i, row in 수리남.iterrows():
    model.eval()

    token_ = tokenizer.encode_plus(
        row['Subtitle'], 
        truncation=True,
        add_special_tokens=True,
        max_length=128,
        padding="max_length",
        return_tensors='pt'
    )

    input_ids = token_['input_ids'].to(device)
    attention_mask = token_['attention_mask'].to(device)

    with torch.no_grad():
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        logits = outputs[0]
        logits = logits.detach().cpu()
        probabilities = torch.sigmoid(logits)
        result = probabilities.squeeze().tolist()

    수리남.loc[i, 'label'] = result[0] 

수리남.head()


,Time,Subtitle,label
0,13s,[불길한 음악],0.009805
1,15s,[부엉이가 운다],0.009847
2,18s,[놀란 숨소리] (인구) 뭐야,0.010115
3,20s,[성난 숨소리],0.010050
4,23s,"어이, 강 선생",0.855419


In [88]:
수리남.to_excel('수리남.xlsx', index=False)

In [ ]:
import pandas as pd
import numpy as np

# Load the Excel data
for i in uploaded.keys():
    마이네임 = pd.read_excel(i)
    마이네임 = 마이네임.drop(['Transliteration'], axis=1)
    마이네임['label'] = np.nan  # Add 'label' column with NaN values

# Iterate over each row and update the 'label' values
for i, row in 마이네임.iterrows():
    model.eval()

    token_ = tokenizer.encode_plus(
        row['Subtitle'], 
        truncation=True,
        add_special_tokens=True,
        max_length=128,
        padding="max_length",
        return_tensors='pt'
    )

    input_ids = token_['input_ids'].to(device)
    attention_mask = token_['attention_mask'].to(device)

    with torch.no_grad():
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        logits = outputs[0]
        logits = logits.detach().cpu()
        probabilities = torch.sigmoid(logits)
        result = probabilities.squeeze().tolist()

    마이네임.loc[i, 'label'] = result[0] 

마이네임.head()


,Time,Subtitle,label
0,6s,"""넷플릭스 시리즈""",0.010129
1,10s,[강렬한 음악],0.010010
2,22s,[어두운 음악],0.010105
3,34s,(필도) 너 이름이 뭐야?,0.010129
4,43s,(필도) 네 진짜 이름이 뭐냐고,0.010157


In [ ]:
마이네임.to_excel('마이네임.xlsx', index=False)

In [98]:
import pandas as pd
import numpy as np

# Load the Excel data
for i in uploaded.keys():
    The_Glory = pd.read_excel(i)
    The_Glory = The_Glory.drop(['Transliteration'], axis=1)
    The_Glory['label'] = np.nan  # Add 'label' column with NaN values

# Iterate over each row and update the 'label' values
for i, row in The_Glory.iterrows():
    model.eval()

    token_ = tokenizer.encode_plus(
        row['Subtitle'], 
        truncation=True,
        add_special_tokens=True,
        max_length=128,
        padding="max_length",
        return_tensors='pt'
    )

    input_ids = token_['input_ids'].to(device)
    attention_mask = token_['attention_mask'].to(device)

    with torch.no_grad():
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        logits = outputs[0]
        logits = logits.detach().cpu()
        probabilities = torch.sigmoid(logits)
        result = probabilities.squeeze().tolist()

    The_Glory.loc[i, 'label'] = result[0] 

The_Glory.head()


,Time,Subtitle,label
0,6s,"""넷플릭스 시리즈""",0.010129
1,10s,[강렬한 음악],0.010010
2,22s,[어두운 음악],0.010105
3,34s,(필도) 너 이름이 뭐야?,0.010129
4,43s,(필도) 네 진짜 이름이 뭐냐고,0.010157


In [90]:
The_Glory.to_excel('더글로리.xlsx', index=False)